# 1. Setup Azure
In this notebook, we will set up the project in Azure by creating the resources we need.

We start by creating the resource group that we'll put all resources into. Then we create our storage account which will be used to store all data (include logs). Finally we set up our Service Bus namespace, and a queue attached to it.

---

### Import packages and load .env

In [1]:
import re
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import os
import json

In [2]:
env_path = find_dotenv()
if env_path=='':
    Path('.env').touch()
    env_path = find_dotenv()
load_dotenv(env_path)

True

### Define variables

Set variables for the project:

- `subscription_id` - the subscription id for your Azure account. Use `az account list -o table` to list all subscriptions
- `resource_group` - the name for the resource group you'll be using for this project. You can think of resource groups as logical containers for the resources you'll create in this tutorial.
- `region` - the region you wish to deploy your resources in:
    - You can see a list of the regions under the key 'name' when running the command `az account list-locations`. 
    - Not all regions support GPU enabled VMs. You can check [here](https://azure.microsoft.com/en-us/pricing/details/virtual-machines/linux/)
    
Set your storage account variables

- `storage_account_name` - the desired name of your storage account
- `storage_container_name` - the desired container name in the storage account is a logical container for individual blobs. For this project, we'll store all content into a single container for ease-of-use
- `model_dir` - the desired name of directory you wish to store your models

Set the Service Bus variables:

- `namespace` - the desired namespace for your Service Bus - this can simply be thought of as a logical container for Service Bus
- `queue` - the desired name of the queue. The queue belongs to the namespace.

Set your docker container variables:
- `docker_login` - the login username for your docker account
- `aks_image_repo` - the desired image repo to use for your aks container
- `aci_image_repo` - the desired image repo to use for your aci container

AKS variables:
- `aks_cluster` - the desired name of the aks cluster

Logic App deployment:
- `logic_app` - the desired name of your logic apps
- `aci_group` - the desired name of your aci group
- `aci_display_name` - the display name used for your aci group

In [3]:
subscription_id = "<your-subscription-id>"           # fill in
resource_group = "<a-resource-group-name>"           # fill in
region = "<selected-region-name>"                    # fill in

storage_account_name = "batchscoringdlsa"            # feel free to replace or use this default
storage_container_name = "aks"                       # feel free to replace or use this default
model_dir = "models"                                 # feel free to replace or use this default

namespace = "batchscoringdlnamespace"                # feel free to replace or use this default
queue = "batchscoringdlqueue"                        # feel free to replace or use this default

docker_login = "<docker-login>"                      # fill in
scoring_image_repo = "batchscoringdl_scoring_app"    # feel free to replace or use this default
flask_image_repo = "batchscoringdl_flask_app"        # feel free to replace or use this default

aks_cluster = "batchscoringdlcluster"                # feel free to replace or use this default
logic_app = "batchscoringdlla"                       # feel free to replace or use this default

In [ ]:
set_key(env_path, "SUBSCRIPTION_ID", subscription_id)
set_key(env_path, "RESOURCE_GROUP", resource_group)
set_key(env_path, "REGION", region)
set_key(env_path, "STORAGE_ACCOUNT_NAME", storage_account_name)
set_key(env_path, "STORAGE_CONTAINER_NAME", storage_container_name)
set_key(env_path, "STORAGE_MODEL_DIR", model_dir)
set_key(env_path, "SB_NAMESPACE", namespace)
set_key(env_path, "SB_QUEUE", queue)
set_key(env_path, "DOCKER_LOGIN", docker_login)
set_key(env_path, "SCORING_IMAGE", scoring_image_repo)
set_key(env_path, "FLASK_IMAGE", flask_image_repo)
set_key(env_path, "AKS_CLUSTER", aks_cluster)
set_key(env_path, "LOGIC_APP", logic_app)

### Set up your resource group
Make sure you've identified which subscription_id and region to use. Create a new resource group to contain all the resources that we create.

This section of the notebook will walk through setting up the resource group using the __az cli__.

In [4]:
!az account set -s {subscription_id}

Create the resource group which we'll put our storage account and all other resources for this project into.

In [ ]:
!az group create -l {region} -n {resource_group}

### Create Azure blob storage
In this section of the notebook, we'll create an Azure blob storage that we'll use throughout the tutorial. This object store will be used to store input and output images as well as any supplementary data such as logs and other scripts that will be used in this workflow.

Use the __az cli__ to create the account

In [5]:
!az storage account create -n {storage_account_name} -g {resource_group} --query 'provisioningState'

"Succeeded" ..


Use the __az cli__ to grab the keys of the storage account that was just created. The `--quote '[0].value'` part of the command simply means to select the _value_ of the _zero-th indexed_ of the set of keys.

In [6]:
key = !az storage account keys list --account-name {storage_account_name} -g {resource_group} --query '[0].value'

The stdout from the command above is stored in a string array of 1. Select the element in the array and ttrip opening and closing quotation marks.

In [7]:
storage_account_key = str(key[0][1:-1]) # this is used to strip opening and closing quotation marks

Use the __az cli__ to create the container in the storage account

In [8]:
!az storage container create \
    --account-name {storage_account_name} \
    --account-key {storage_account_key} \
    --name {storage_container_name}

{
  "created": true
}


Set storage account key to dotenv file.

In [9]:
set_key(env_path, "STORAGE_ACCOUNT_KEY", storage_account_key) # generated

### Create Service Bus & Generated Keys

Create the namespace.

In [10]:
!az servicebus namespace create \
    --resource-group {resource_group} \
    --name {namespace} 

Create a service bus queue. Set the lock duration to 5 minutes. This means that the lock for each queue message will last for 5 minutes.

In [11]:
!az servicebus queue create \
    --resource-group {resource_group} \
    --namespace-name {namespace} \
    --name {queue} \
    --lock-duration PT5M

Now that we've created our Service Bus namespace and queue, we need to get the key-name/key-value pair so that we can access it.

By default, your Service Bus resource will come with an key-value "authorization rule" pair - its key name will have the value: "RootManageSharedAccessKey". The following command will get the key name for the "authorization rule", and assign it as `sb_key_name`.

In [12]:
sb_key_name = !az servicebus namespace authorization-rule list \
    --resource-group {resource_group} \
    --namespace-name {namespace} \
    -o json --query "[0].name"

sb_key_name = str(sb_key_name[0][1:-1])

Get the primary key value to "RootManageSharedAccessKey".

In [13]:
sb_credentials = !az servicebus namespace authorization-rule keys list \
    --resource-group {resource_group} \
    --namespace-name {namespace} \
    --name {sb_key_name} \
    -o json --query "primaryKey"

In [14]:
sb_key_value = re.findall(r'"(.*?)"', str(sb_credentials))[0]

Set the service bus key value to the dotenv file.

In [15]:
set_key(env_path, "SB_SHARED_ACCESS_KEY_VALUE", sb_key_value) # generated
set_key(env_path, "SB_SHARED_ACCESS_KEY_NAME", sb_key_name) # generated

---

### Set environment variables to be used by later notebooks

Check that our `.env` file looks correct.

In [16]:
!cat .env

### Set up Azure Blob as a File System on our local machine using Blobfuse

Install blob fuse

In [17]:
ubuntu_version = !lsb_release -r | grep -o '[0-9][0-9].[0-9][0-9]'

In [18]:
!wget https://packages.microsoft.com/config/ubuntu/{ubuntu_version[0]}/packages-microsoft-prod.deb
!sudo dpkg -i packages-microsoft-prod.deb
!sudo apt-get update

--2019-01-22 16:59:27--  https://packages.microsoft.com/config/ubuntu/18.04/packages-microsoft-prod.deb
Resolving packages.microsoft.com (packages.microsoft.com)... 40.76.35.62
Connecting to packages.microsoft.com (packages.microsoft.com)|40.76.35.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3132 (3.1K) [application/octet-stream]
Saving to: ‘packages-microsoft-prod.deb’

packages-microsoft- 100%[===================>]   3.06K  --.-KB/s    in 0s      

2019-01-22 16:59:27 (664 MB/s) - ‘packages-microsoft-prod.deb’ saved [3132/3132]

(Reading database ... 233115 files and directories currently installed.)
Preparing to unpack packages-microsoft-prod.deb ...
Unpacking packages-microsoft-prod (1.0-ubuntu18.04.2) over (1.0-ubuntu18.04.2) ...
Setting up packages-microsoft-prod (1.0-ubuntu18.04.2) ...
Hit:1 http://azure.archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://azure.archive.ubuntu.com/ubuntu bionic-updates InRelease          
Get:3 http://azure.arc

In [19]:
!sudo apt-get install blobfuse

Reading package lists... Done
Building dependency tree       
Reading state information... Done
blobfuse is already the newest version (1.0.2).
The following packages were automatically installed and are no longer required:
  linux-azure-cloud-tools-4.15.0-1031 linux-azure-cloud-tools-4.15.0-1032
  linux-azure-headers-4.15.0-1031 linux-azure-headers-4.15.0-1032
  linux-azure-tools-4.15.0-1031 linux-azure-tools-4.15.0-1032
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 114 not upgraded.


Setup ramdisk tmp dir for blob fuse for low latency buffer.

In [20]:
!sudo rm -rf /mnt/ramdisk && sudo mkdir /mnt/ramdisk
!sudo mount -t tmpfs -o size=4g tmpfs /mnt/ramdisk
!sudo mkdir /mnt/ramdisk/blobfusetmp
!sudo chown $(whoami) /mnt/ramdisk/blobfusetmp

rm: cannot remove '/mnt/ramdisk': Device or resource busy


Setup credentials for blob connection

In [21]:
!rm -f fuse_connection.cfg
!touch fuse_connection.cfg
!echo accountName {get_key(env_path, "STORAGE_ACCOUNT_NAME")} >> fuse_connection.cfg
!echo accountKey {get_key(env_path, "STORAGE_ACCOUNT_KEY")} >> fuse_connection.cfg
!echo containerName {get_key(env_path, "STORAGE_CONTAINER_NAME")} >> fuse_connection.cfg
!chmod 700 fuse_connection.cfg

Create mount directory

In [22]:
!rm -rf data && mkdir data

Mount blob to mount directory

In [23]:
!blobfuse data \
    --tmp-path=/mnt/ramdisk/blobfusetmp  \
    --config-file=fuse_connection.cfg \
    -o attr_timeout=240 \
    -o entry_timeout=240 \
    -o negative_timeout=120

### Add Model Dir and Video to storage

Copy `models` dir to the `data` folder we just mounted.

In [24]:
!cp -r -n models data

Download an `orangutan.mp4` video to use throughout the tutorial.

In [25]:
!wget https://happypathspublic.blob.core.windows.net/videos/orangutan.mp4

--2019-01-22 16:59:34--  https://happypathspublic.blob.core.windows.net/videos/orangutan.mp4
Resolving happypathspublic.blob.core.windows.net (happypathspublic.blob.core.windows.net)... 52.239.214.164
Connecting to happypathspublic.blob.core.windows.net (happypathspublic.blob.core.windows.net)|52.239.214.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7961293 (7.6M) [video/mp4]
Saving to: ‘orangutan.mp4’

orangutan.mp4       100%[===================>]   7.59M  --.-KB/s    in 0.05s   

2019-01-22 16:59:34 (140 MB/s) - ‘orangutan.mp4’ saved [7961293/7961293]



Move that video into the mounted `data` folder.

In [26]:
!mv orangutan.mp4 data 

Check that our models dir and orangutan.mp4 is uploaded to our mounted blob storage container.

In [27]:
!ls data

models	orangutan.mp4


Continue to the next [notebook](/notebooks/02_local_testing.ipynb).